**Pedro Paulo da Costa Pereira - A88062**<br>
**Tiago André Oliveira Leite - A91693**

# <center>TP1 Logica Computacional</center>

## Problema 2 - Sudoku
Da definição do jogo “Sudoku” generalizado para a dimensão $N$; o problema tradicional corresponde ao caso $N=3$. O objetivo do Sudoku é preencher uma grelha de $\,N^2\times N^2\,$ com inteiros positivos no intervalo $\,1$ até $\,N^2\,$, satisfazendo as seguintes regras:<br><br>
        - Cada inteiro no intervalo $\,1$ até $\,N^2\,$ocorre  só uma vez em cada coluna, linha e secção $\,N\times N\,$.<br><br>
        - No início do jogo uma fração $\,0\leq 𝛼< 1\,$ das $\,N^4\,$ casas da grelha são preenchidas de forma consistente com a regra anterior. 

### Variaveis do Problema
Para resolver o problema foram utilizadas $\,N^2\times N^2\times N^2\,$ variavies binarias, que nos premite verificar qual o número que vai ser colocado numa determinada linha e coluna da grelja do sudoku.<br>
Assim sendo foi definido o seguinte grupo de variavies:<br>
- $\quad cube_{row,col,depth}$ - variavel que represena o número $depth + 1$ na linha $row$ e na coluna $col$.


### Restrições
- $\forall_{row,col,row}$  $cube[(row,col,depth)] \in \{0,1\}$<br>
- $\forall_{row,col}\sum_{depth}$ $cube[(row,col,depth)] == 1$<br>
- $\forall_{row,depth}\sum_{col}$ $cube[(row,col,depth)] == 1$<br>
- $\forall_{depth,col}\sum_{row}$ $cube[(row,col,depth)] == 1$

Seja $n$ uma secção  $\,N\times N\,$ do cubo: <br>
- $\forall_{n_{row},n_{col}}\sum_{depth}$ $cube[(n_{row},n_{col},depth)] == 1$<br>
- $\forall_{n_{row},depth}\sum_{n_{row}}$ $cube[(n_{row},n_{col},depth)] == 1$<br>
- $\forall_{depth,n_{col}}\sum_{n_{row}}$ $cube[(n_{row},n_{col},depth)] == 1$



In [9]:
import math
import random
import timeit
import time
from ortools.linear_solver import pywraplp
import matplotlib.pyplot as plt

### Exemplo de sudoku

### Função que imprime a grelha do sudoku

In [10]:
def print_sudoku(sudoku):
    grid = 0 
    l = len(sudoku)
    n = int(math.sqrt(l))
    pad = 1 + l // 10
    r = (l + n) * (pad + 1)  +1
    if n > 4 and not grid:
        for i in range(l):
            for j in range(l):
                print(f'{sudoku[i][j]}'.ljust(pad), end = " ")
            print('')
        return 
                
    for i in range(l):
        if i % n == 0:
            print("-"*r)
        for j in range(l):
            if  j % n == 0:
                print("|".ljust(pad), end = " ")
            print(f'{sudoku[i][j]}'.ljust(pad), end = " ")
            if  j == l-1:
                print("|".ljust(pad), end = " ")
        print('')
    print("-" * r)

### Função auxiliar que converte o resultado do solver numa matriz

In [11]:
def converter(cube,dim):
    mat = [[0 for a in range(dim)] for b in range(dim) ]
    for row in range(dim):
        for col in range(dim):
            for depth in range(dim):
                if cube[(row,col,depth)].solution_value() == 1:
                    mat[row][col] = depth + 1
    return mat

### Função que inicializa a grelha do sudoku
Recebe como parametros a dimensão $N$ e a fracção $0 ≤ α < 1$ de casas que sao preenchidas. Para tal coloca $N$ números aletorios e depois tenta resolver o sudoku. De seguida apaga o número de números necessários para satisfazer α

In [12]:
def sudoku_generator(N, alpha):
    dim = N*N
    delete = int(dim * dim * (1-alpha))
    num_squares = int(math.sqrt(dim))
    solver = pywraplp.Solver.CreateSolver('SCIP')
    cube = {}
    
    
    for row in range(dim):
        for col in range(dim):
            for depth in range(dim):
                cube[(row,col,depth)] = solver.BoolVar('%i%i%i' % (row,col,depth))

    for row in range(dim):
        for col in range(dim):
            val = []
            for depth in range(dim):
                val.append(cube[(row,col,depth)])
            solver.Add(sum(val) == 1)

    for row in range(dim):
        for depth in range(dim):
            val = []
            for col in range(dim):
                val.append(cube[(row,col,depth)])
            solver.Add(sum(val) == 1)

    for depth in range(dim):
        for col in range(dim):
            val = []
            for row in range(dim):
                val.append(cube[(row, col, depth)])
            solver.Add(sum(val) == 1)

    for i in range(dim):
        corner_y = i - i % N
        for j in range(dim):
            corner_x = j - j % N
            for depth in range(dim):
                val = []
                for row in range(N):
                    for col in range(N):
                        val.append(cube[corner_y + row, corner_x + col, depth])
                solver.Add(sum(val) == 1)

    for i in range(N):
        randoms = []
        row = random.randint(0,dim-1)
        col = random.randint(0,dim-1)
        depth = random.randint(1, dim - 1)
        while (row,col) in randoms:
            row = random.randint(0,dim-1)
            col = random.randint(0,dim-1)
        solver.Add(cube[(row,col,depth)] == 1)
    
    status = solver.Solve()
    if status == pywraplp.Solver.OPTIMAL:
        mat = converter(cube, dim)
        while(delete > 0):
            row = random.randint(0,dim-1)
            col = random.randint(0,dim-1)
            while mat[row][col] == 0:
                row = random.randint(0, dim-1)
                col = random.randint(0, dim-1)
            mat[row][col] = 0
            delete -=1
        
        return mat
    else:
        return False

In [13]:
mat = sudoku_generator(3,0.4)
if mat:
    print_sudoku(mat)


-------------------------
| 0 9 0 | 7 0 0 | 0 3 0 | 
| 0 5 4 | 2 0 3 | 9 6 0 | 
| 0 0 0 | 0 4 1 | 8 0 5 | 
-------------------------
| 0 7 0 | 0 5 2 | 0 0 0 | 
| 0 0 0 | 0 1 9 | 0 0 0 | 
| 5 0 0 | 0 3 0 | 0 4 0 | 
-------------------------
| 4 8 0 | 0 7 6 | 1 0 3 | 
| 0 6 0 | 0 2 0 | 0 0 4 | 
| 0 0 0 | 3 0 4 | 0 0 6 | 
-------------------------


### Outra forma de inicializar o sudoku mas que produz sudokus impossiveis

In [15]:
def check(sudoku, row, column, number):
    if number in sudoku[row]:
        return False
    for r in sudoku:
        if r[column] == number:
            return False
    n = int(math.sqrt(len(sudoku)))
    r = (row // n) * n
    c = (column // n) * n
    for i in range(n):
        for j in range(n):
            if sudoku[r+i][c+j] == number:
                return False
    return True

def sudoku_generator_simple(N, alpha):
    dim = N*N
    filled = int(dim * dim * alpha)
    mat = [[0 for a in range(dim)] for a in range(dim)]
    while(filled > 0):
        row = random.randint(0,dim-1)
        col = random.randint(0,dim-1)
        number = random.randint(1, dim - 1)
        while(mat[row][col] != 0 or not check(mat,row,col,number)):
            row = random.randint(0, dim-1)
            col = random.randint(0, dim-1)
            number = random.randint(1,dim-1)
        mat[row][col] = number
        filled -=1

    return mat

In [16]:
mat = sudoku_generator_simple(5,0.2)
if mat:
    print_sudoku(mat)

0   0   0   0   0   4   0   0   1   0   0   0   3   0   0   0   0   0   12  0   0   0   0   0   0   
0   0   0   11  0   0   0   0   0   0   17  0   5   0   6   0   0   0   0   0   0   0   8   0   12  
0   0   0   0   0   0   21  0   0   22  0   0   7   0   0   0   0   20  10  1   0   0   19  0   0   
0   6   0   16  0   0   0   0   9   0   0   0   0   0   0   0   0   0   3   0   0   0   22  5   0   
0   0   21  0   0   0   0   0   0   0   0   0   0   0   0   0   0   16  0   14  0   0   0   0   0   
0   0   0   24  0   0   0   0   0   0   0   0   0   0   0   8   14  0   0   0   5   0   23  0   20  
0   0   0   0   7   0   0   0   0   18  0   0   0   15  0   0   0   0   0   0   8   0   0   3   14  
15  0   0   0   0   0   0   19  0   0   0   0   0   0   0   0   1   0   0   0   0   0   0   0   4   
0   0   0   21  0   0   24  0   0   0   0   0   0   0   0   23  0   0   0   0   0   0   10  0   0   
0   22  1   2   0   0   0   9   14  8   0   0   0   0   0   24  0   0   15  0   0   0   0  

### Função que resolve o sudoku

In [8]:
def sudoku_solver(sudoku):
    solver = pywraplp.Solver.CreateSolver('SCIP')
    dim = len(sudoku)
    num_squares = int(math.sqrt(dim))
    cube = {}

    for row in range(dim):
        for col in range(dim):
            for depth in range(dim):
                cube[(row,col,depth)] = solver.BoolVar('%i%i%i' % (row,col,depth))


    for row in range(dim):
        for col in range(dim):
            val = []
            for depth in range(dim):
                val.append(cube[(row,col,depth)])
            solver.Add(sum(val) == 1)

    for row in range(dim):
        for depth in range(dim):
            val = []
            for col in range(dim):
                val.append(cube[(row,col,depth)])
            solver.Add(sum(val) == 1)

    for depth in range(dim):
        for col in range(dim):
            val = []
            for row in range(dim):
                val.append(cube[(row, col, depth)])
            solver.Add(sum(val) == 1)

    for i in range(dim):
        corner_y = i - i % num_squares
        for j in range(dim):
            corner_x = j - j % num_squares
            for depth in range(dim):
                val = []
                for row in range(num_squares):
                    for col in range(num_squares):
                        val.append(cube[corner_y + row, corner_x + col, depth])
                solver.Add(sum(val) == 1)

    for i in range(dim):
        for j in range(dim):
            if sudoku[i][j] != 0:
                solver.Add(cube[(i,j,sudoku[i][j]-1)] == 1)


    status = solver.Solve()
    mat = converter(cube, dim)
    if status == pywraplp.Solver.OPTIMAL:
        return mat
    else:
        return False

### Testes

In [23]:
def test_sudoku(N, alpha):
    mat = sudoku_generator(N, alpha)
    print_sudoku(mat)
    print("")
    mat = sudoku_solver(mat)
    if mat:
        print_sudoku(mat)
    

In [32]:
times = []

### $N = 3$<br>$\alpha = 0.5$

In [33]:
times.append((timeit.timeit(stmt='test_sudoku(3,0.0)', setup='from __main__ import test_sudoku',number=3 ))/3)

-------------------------
| 3 4 1 | 9 2 6 | 8 5 7 | 
| 5 7 6 | 3 4 8 | 2 9 1 | 
| 8 9 2 | 5 1 7 | 3 4 6 | 
-------------------------
| 1 2 5 | 8 7 4 | 9 6 3 | 
| 7 3 9 | 2 6 5 | 1 8 4 | 
| 4 6 8 | 1 3 9 | 7 2 5 | 
-------------------------
| 9 8 7 | 6 5 3 | 4 1 2 | 
| 6 1 3 | 4 8 2 | 5 7 9 | 
| 2 5 4 | 7 9 1 | 6 3 8 | 
-------------------------

-------------------------
| 3 4 1 | 9 2 6 | 8 5 7 | 
| 5 7 6 | 3 4 8 | 2 9 1 | 
| 8 9 2 | 5 1 7 | 3 4 6 | 
-------------------------
| 1 2 5 | 8 7 4 | 9 6 3 | 
| 7 3 9 | 2 6 5 | 1 8 4 | 
| 4 6 8 | 1 3 9 | 7 2 5 | 
-------------------------
| 9 8 7 | 6 5 3 | 4 1 2 | 
| 6 1 3 | 4 8 2 | 5 7 9 | 
| 2 5 4 | 7 9 1 | 6 3 8 | 
-------------------------
-------------------------
| 2 1 3 | 4 5 6 | 7 8 9 | 
| 4 6 7 | 1 8 9 | 2 5 3 | 
| 5 8 9 | 2 3 7 | 1 6 4 | 
-------------------------
| 1 2 4 | 3 9 8 | 6 7 5 | 
| 3 7 5 | 6 1 2 | 4 9 8 | 
| 6 9 8 | 5 7 4 | 3 2 1 | 
-------------------------
| 7 3 1 | 9 6 5 | 8 4 2 | 
| 8 5 2 | 7 4 1 | 9 3 6 | 
| 9 4 6 | 8

In [40]:
times[0]

0.1137659060001776

### $N = 4$<br>$\alpha = 0.3$

In [35]:
times.append((timeit.timeit(stmt='test_sudoku(4,0.2)', setup='from __main__ import test_sudoku',number=3 ))/3)

-------------------------------------------------------------
|  16 2  8  6  |  5  0  14 9  |  0  0  11 13 |  1  0  0  15 |  
|  0  9  0  5  |  7  12 15 4  |  0  8  0  2  |  11 0  13 14 |  
|  12 0  15 0  |  11 1  6  8  |  16 14 7  0  |  0  10 9  2  |  
|  7  4  0  11 |  0  0  2  0  |  5  1  9  15 |  0  0  8  3  |  
-------------------------------------------------------------
|  6  14 11 4  |  13 16 1  15 |  9  5  3  0  |  12 2  7  10 |  
|  8  7  9  0  |  4  0  5  3  |  0  2  1  12 |  16 11 6  13 |  
|  2  10 16 13 |  0  7  9  14 |  4  6  0  11 |  3  1  5  8  |  
|  0  1  3  12 |  8  2  11 6  |  13 16 0  7  |  15 4  14 0  |  
-------------------------------------------------------------
|  14 16 13 1  |  2  4  3  12 |  8  15 5  9  |  7  6  10 0  |  
|  0  8  0  2  |  0  5  10 11 |  6  12 13 1  |  4  9  0  16 |  
|  11 5  6  9  |  0  8  16 1  |  10 7  4  3  |  13 0  0  12 |  
|  15 12 0  0  |  9  6  7  13 |  2  11 16 14 |  8  5  3  1  |  
----------------------------------------------

In [41]:
times[1]

3.3533036086667685

### $N = 5$<br>$\alpha = 0.2$

In [38]:
times.append(timeit.timeit(stmt='test_sudoku(5,0.4)', setup='from __main__ import test_sudoku',number=1 ))

7   2   25  23  19  15  22  10  0   0   0   0   12  5   0   13  0   0   11  4   0   8   0   0   24  
0   0   3   6   15  7   0   11  0   17  0   25  0   0   2   0   0   24  0   22  4   23  0   0   1   
4   9   0   1   22  0   2   20  23  5   11  0   24  7   19  25  0   0   0   18  10  6   0   13  14  
0   0   21  10  24  0   18  12  0   8   9   0   3   23  6   7   0   14  2   0   0   25  15  0   11  
17  0   11  0   14  0   9   3   25  0   4   0   8   10  0   19  0   5   15  0   7   12  22  0   0   
20  0   24  0   12  18  16  19  0   0   10  2   1   22  0   15  5   25  0   17  3   13  0   6   8   
11  0   0   9   0   0   0   0   12  1   15  3   0   0   13  24  22  0   6   0   21  5   0   0   2   
0   19  0   0   6   0   20  0   11  0   0   0   0   9   7   1   2   13  0   0   15  24  23  0   0   
0   0   0   3   13  22  0   0   0   0   17  0   19  0   21  0   0   0   9   0   0   0   4   25  0   
8   18  1   25  0   3   0   0   0   13  0   0   23  16  0   14  4   21  19  0   12  22  0  

In [42]:
times[2]

288.8575999910008

### Tempos de Execução

In [44]:
times

[0.1137659060001776, 3.3533036086667685, 288.8575999910008]

Para $N = 5$ a função demora varios minutos a executar, o que deve ser provocado pelo ao facto de as soluções terem complexidade exponencial em $N$. Neste caso, o problema vai ter $\,5^2\times 5^2\times 5^2\, = \,25^3\, = 15625$ variaveis.